#**Proyecto de Visualización de Datos**

---



**Integrantes:**

* Frank Ygnacio
* Ricardo Llanos
* Yulian Cama
* Fiorela Lizarraga

---


## 1. Importación de Librerias

In [ ]:
#importamos librerías principales
import numpy as np
import pandas as pd

#importamos librerías esenciales para la visualización
from bokeh.palettes import PRGn, RdYlGn
from bokeh.transform import linear_cmap,factor_cmap
from bokeh.models import ColorBar, NumeralTickFormatter, HoverTool, CustomJS, RangeSlider, TextInput, RadioButtonGroup
from bokeh.tile_providers import CARTODBPOSITRON,get_provider, Vendors
from bokeh.plotting import figure, show, output_notebook, save, ColumnDataSource
from bokeh.layouts import layout, column, row

## 2. Importación de Datos

In [ ]:
#abrimos el conjunto de datos
data = pd.read_csv("earthquakes.csv")

#visualizamos algunos ejemplos
data.head()

,Year,Mo,Dy,Hr,Mn,Sec,Tsu,Vol,Location Name,Latitude,Longitude,Focal Depth (km),Mag,MMI Int,Deaths,Death Description,Missing,Missing Description,Injuries,Injuries Description,Damage ($Mil),Damage Description,Houses Destroyed,Houses Destroyed Description,Houses Damaged,Houses Damaged Description,Total Deaths,Total Death Description,Total Missing,Total Missing Description,Total Injuries,Total Injuries Description,Total Damage ($Mil),Total Damage Description,Total Houses Destroyed,Total Houses Destroyed Description,Total Houses Damaged,Total Houses Damaged Description
0,-2150.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,"JORDAN: BAB-A-DARAA,AL-KARAK",31.1,35.5,NaN,7.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TURKMENISTAN: W,38.0,58.2,18.0,7.1,10.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN
2,-1250.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,ISRAEL: ARIHA (JERICHO),32.0,35.5,NaN,6.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1050.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,JORDAN: SW: TIMNA COPPER MINES,29.6,35.0,NaN,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-479.0,NaN,NaN,NaN,NaN,NaN,9.0,NaN,GREECE: MACEDONIA,39.7,23.3,NaN,7.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Preparación de la data

In [ ]:
#asignamos las columnas necesarias en el dataframe preparado
data_sample = data
data_sample = data_sample[["Year","Location Name","Mag","MMI Int","Latitude", "Longitude","Total Deaths"]]

In [ ]:
data_sample['Location Name'] = data_sample['Location Name'].

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [ ]:


#completamos los nan con el valor de 0
data_sample = data_sample.fillna(value=0)

#actualizamos las columnas
data_sample.columns = ["year","location","magnitude","mmi","latitude", "longitude","deaths"]

#añadimos el tamaño del circulo que representará el impacto en vidas
total_deaths = data_sample.deaths.sum()
print(total_deaths)
print(data_sample.deaths.max())
circle_size = 10
size = 100
data_sample["size"] = circle_size + ((data_sample.deaths/total_deaths)*size if total_deaths != 0 else 0)
#añadimos el tamaño del circulo que representará la intensidad del terremoto en MMI
circlemmi_size = 5
data_sample["sizemmi"] = circlemmi_size + (data_sample.mmi)
#visualizamos algunos ejemplos
data_sample.head()

5564150.0
830000.0


,year,location,magnitude,mmi,latitude,longitude,deaths,size,sizemmi
0,-2150.0,"JORDAN: BAB-A-DARAA,AL-KARAK",7.3,0.0,31.1,35.5,0.0,10.000000,5.0
1,-2000.0,TURKMENISTAN: W,7.1,10.0,38.0,58.2,1.0,10.000018,15.0
2,-1250.0,ISRAEL: ARIHA (JERICHO),6.5,0.0,32.0,35.5,0.0,10.000000,5.0
3,-1050.0,JORDAN: SW: TIMNA COPPER MINES,6.2,0.0,29.6,35.0,0.0,10.000000,5.0
4,-479.0,GREECE: MACEDONIA,7.0,9.0,39.7,23.3,0.0,10.000000,14.0


## 4. Transformación de Lat/Long a Mercator Coords.

Bokeh utiliza el [**Sistema de Coordenadas Universal Transversal Mercator**](https://es.wikipedia.org/wiki/Sistema_de_coordenadas_universal_transversal_de_Mercator). Por tanto, no podemos utilizar los valores de Lat/Long para la localización. Sin embargo, estos serán útiles para la referencia en la colocación de puntos. Por tanto, debemos computar el proceso de transformación de valores latitudinales y longitudinales a coordenadas Mercator.

In [ ]:
# Función de transformación de valores de lat/long a Mercator
def x_coord(x, y):
    """
    Toma como parámetros de entrada los valores de Latitud(x) y Longitud (y).
    Devuelve una tupla con las coordenadas Mercator equivalentes.
    
    Estas deberán ser almacenadas como una tupla en el pd.DataFrame org.
    """
    #definición lat/long
    lat = x
    lon = y
    
    #mavor r_major de transformación
    r_major = 6378137.000
    
    #nueva "latitud" mercator
    x = r_major * np.radians(lon)
    
    #escala de transformación
    scale = x/lon
    
    #nueva "longitud" mercator
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    
    #coordenadas finales (tuple)
    return (x, y)

In [ ]:
#computamos coordenadas mercator utilizando Latitude & Longitude
data_sample["mercator"]= data_sample.apply(
    lambda x: x_coord(x.latitude, x.longitude), 
    axis=1
)

#separamos las coordenadas mercator en X e Y
data_sample[['mercator_x', 'mercator_y']] = data_sample['mercator'].apply(pd.Series)

#visualizamos algunos ejemplos
data_sample

,year,location,magnitude,mmi,latitude,longitude,deaths,size,sizemmi,mercator,mercator_x,mercator_y
0,-2150.0,"JORDAN: BAB-A-DARAA,AL-KARAK",7.3,0.0,31.100,35.500,0.0,10.000000,5.0,"(3951841.9231612114, 3645742.8680035304)",3.951842e+06,3.645743e+06
1,-2000.0,TURKMENISTAN: W,7.1,10.0,38.000,58.200,1.0,10.000018,15.0,"(6478794.364168522, 4579425.812870097)",6.478794e+06,4.579426e+06
2,-1250.0,ISRAEL: ARIHA (JERICHO),6.5,0.0,32.000,35.500,0.0,10.000000,5.0,"(3951841.9231612114, 3763310.627144652)",3.951842e+06,3.763311e+06
3,-1050.0,JORDAN: SW: TIMNA COPPER MINES,6.2,0.0,29.600,35.000,0.0,10.000000,5.0,"(3896182.1777645745, 3452236.5008381675)",3.896182e+06,3.452237e+06
4,-479.0,GREECE: MACEDONIA,7.0,9.0,39.700,23.300,0.0,10.000000,14.0,"(2593744.1354832742, 4822442.387442775)",2.593744e+06,4.822442e+06
...,...,...,...,...,...,...,...,...,...,...,...,...
4419,2020.0,PHILIPPINES: MASBATE,6.6,7.0,12.021,124.123,1.0,10.000018,12.0,"(13817309.155733498, 1348098.436685851)",1.381731e+07,1.348098e+06
4420,2020.0,ALASKA,7.6,8.0,54.608,-159.655,0.0,10.000000,13.0,"(-17772713.30260009, 7286155.599177742)",-1.777271e+07,7.286156e+06
4421,2020.0,GREECE: SAMOS; TURKEY: IZMIR,7.0,8.0,37.918,26.790,118.0,10.002121,13.0,"(2982249.158351799, 4567848.430509404)",2.982249e+06,4.567848e+06
4422,2020.0,CHILE: OFF COAST CENTRAL,6.7,5.0,-39.343,-74.990,0.0,10.000000,10.0,"(-8347848.614587584, -4770923.150547308)",-8.347849e+06,-4.770923e+06


## 5. Visualización en Bokeh

In [ ]:
#definimos proveedor de mapa base
chosentile = get_provider(CARTODBPOSITRON)

#definimos fuente de información (nuestro dataframe de trabajo)
s1 = ColumnDataSource(data=data_sample) #[data_sample['Mag']>=8]
s2 = ColumnDataSource(data=data_sample)
#definimos una paleta de colores cualesquiera
palette = RdYlGn[10] 

#definimos el rango de coloración del mapa con base a una variable (magnitud del sismo)
color_mapper = linear_cmap(
    field_name = 'magnitude', #Magnitud del sismo
    palette = palette, low = data_sample['magnitude'].min(), 
    high = data_sample['magnitude'].max()
)

#definimos el tooltip para manejar información adicional en el mapa
TOOLTIPS = [
    ("year", "@year"),
    ("location", "@location"),
    ("(lat,lon)", "(@latitude, @longitude)"),
    ("mag", "@magnitude{0,0.0}"),
    ("deaths","@deaths")
]

#Definimos barra de color para identificar rango de valores de intensidad de Mag.
color_bar = ColorBar(
    color_mapper=color_mapper['transform'], 
    formatter = NumeralTickFormatter(format='0.0[0000]'), 
    label_standoff = 13, width=5, location=(0,0)
)

In [ ]:
#definimos las variables de información de nuestro 'p' (plot)
p = figure(
    title = 'Terremotos en el Mundo', 
    x_axis_type= "mercator", 
    y_axis_type= "mercator", 
    x_axis_label= "longitude", 
    y_axis_label= "latitude",
    tooltips=TOOLTIPS,
    plot_width=1000, 
    plot_height=600
)

#añadimos el mapa base seleccionado
p.add_tile(chosentile)

#posición de la barra (a la derecha del plot)
p.add_layout(color_bar, 'right')

# Añadimos los puntos de las variables de información (Mag) según coords. mercator
p.circle(
    x = 'mercator_x', 
    y = 'mercator_y',    
    size='size', 
    source=s2, 
    #color = color_mapper, 
    fill_color = color_mapper,
    line_color = color_mapper,
    fill_alpha = 0.25
)

callback_slider = CustomJS(args=dict(s1=s1,s2=s2),
                    code="""
    var data1 = s1.data;
    var data2 = s2.data;

    var start = cb_obj.value[0];
    var end = cb_obj.value[1];
    var ind = cb_obj.indices;

    for ( var i = 0; i < data1['index'].length; i++){
        if (data1['year'][i]>=start && data1['year'][i]<=end){
          data2['size'][i] = data1['size'][i];
        }
        else {
          data2['size'][i] = 0;
        }
    }
    

    s2.change.emit();
""")

callback_textinput = CustomJS(args=dict(s1=s1,s2=s2),
                    code="""
    var data1 = s1.data;
    var data2 = s2.data;

    var country = cb_obj.value;
    var ind = cb_obj.indices;
    
    for ( var i = 0; i < data1['index'].length; i++){
        if (data1['location'][i].includes(country)){
          data2['size'][i] = data1['size'][i];
        }
        else {
          data2['size'][i] = 0;
        }
    }
    

    s2.change.emit();
""")

year_slider = RangeSlider(start=1550, end=2021, value=(1550,1600), step=1, title="Rango de años", bar_color = 'darkgray')
year_slider.js_on_change('value', callback_slider)

textinput = TextInput(value="",title = "Nombre del País contiene")
textinput.js_on_change('value',callback_textinput)

layout = row(column(textinput, year_slider), p)

#solicitamos el plot en el notebook
output_notebook()

#plot final
show(layout)

#### MMI

In [ ]:
data_mmi = data_sample.sort_values(by="mmi", ignore_index=True)
# data_mmi.head()

,year,location,magnitude,mmi,latitude,longitude,deaths,size,sizemmi,mercator,mercator_x,mercator_y
0,-2150.0,"JORDAN: BAB-A-DARAA,AL-KARAK",7.3,0.0,31.100,35.500,0.0,10.000000,5.0,"(3951841.9231612114, 3645742.8680035304)",3.951842e+06,3.645743e+06
1,1988.0,"TAIWAN: HUALIEN, SU-HUA",5.7,0.0,23.902,121.598,1.0,10.000018,5.0,"(13536227.44148048, 2741470.921621218)",1.353623e+07,2.741471e+06


In [ ]:
#definimos proveedor de mapa base
chosentile = get_provider(CARTODBPOSITRON)

#definimos fuente de información (nuestro dataframe de trabajo)
s1 = ColumnDataSource(data=data_mmi) 
s2 = ColumnDataSource(data=data_mmi)
#definimos una paleta de colores cualesquiera
palette = RdYlGn[10]

#definimos el rango de coloración del mapa con base a una variable (mmi)
color_mapper = linear_cmap(
    field_name = 'mmi', #MMI
    palette = palette, low = data_mmi['mmi'].min(), 
    high = data_mmi['mmi'].max()
)

#definimos el tooltip para manejar información adicional en el mapa
TOOLTIPS = [
    ("year", "@year"),
    ("location", "@location"),
    ("(lat,lon)", "(@latitude, @longitude)"),
    ("mmi","@mmi"),
    ("mag", "@magnitude{0,0.0}")
]

#Definimos barra de color para identificar rango de valores de la escala de Mercalli.
color_bar = ColorBar(
    color_mapper=color_mapper['transform'], 
    formatter = NumeralTickFormatter(format='0.0[0000]'), width=5, location=(0,0)
)

In [ ]:
p = figure(
    title = 'MMI de los Terremotos en el Mundo', 
    x_axis_type= "mercator", 
    y_axis_type= "mercator", 
    x_axis_label= "longitude", 
    y_axis_label= "latitude",
    tooltips=TOOLTIPS,
    plot_width=1000, 
    plot_height=600
)

#añadimos el mapa base seleccionado
p.add_tile(chosentile)

#posición de la barra (a la derecha del plot)
p.add_layout(color_bar, 'right')

# Añadimos los puntos de las variables de información (Mag) según coords. mercator
p.circle(
    x = 'mercator_x', 
    y = 'mercator_y',    
    size='sizemmi', 
    source=s2, 
    #color = color_mapper, 
    fill_color = color_mapper,
    line_color = color_mapper,
    fill_alpha = 0.25
)

mmi_list = list(map(str,range(2,13)))

r_b_g = CustomJS(args=dict(s1=s1,s2=s2),
                    code="""
    var data1 = s1.data;
    var data2 = s2.data;

    var A = cb_obj.active;
    var ind = cb_obj.indices;

    for ( var i = 0; i < data1['index'].length; i++){
        if (data1['mmi'][i]==A+2){
          data2['sizemmi'][i] = data1['sizemmi'][i];
        }
        else {
          data2['sizemmi'][i] = 0;
        }
    }    

    s2.change.emit();
""")


rb_group = RadioButtonGroup(labels=mmi_list)
rb_group.js_on_change('active', r_b_g)

layout = column(rb_group, p)

#solicitamos el plot en el notebook
output_notebook()

#plot final
show(layout)